In [236]:
import nltk
from nltk.tokenize import RegexpTokenizer, wordpunct_tokenize
from nltk.tokenize import sent_tokenize
import operator
import os
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import statistics 
from statistics import mean
from functools import reduce
from decimal import Decimal
import pandas as pd
#import random


In [237]:
files= os.listdir('brown_train') #get all file names under the training dataset
#all_text = []
all_text2 = ''
for file in files: 
    f = open('brown_train/'+file, mode = 'r')
    article = f.read()
    #all_text.append(article.lower()) # lowercase all the words
    all_text2 = all_text2 + article.lower()
    f.close()

In [272]:
def token(text):
    temp = text.split()
    
    return temp

def split_word_tag(text_token):
    splited = []
    for j in text_token:
        temptemp = j.split('/')
        splited.append(temptemp)
    return splited

In [284]:
def add_START_END(word_tag_list):
    
    for i in range(len(word_tag_list)):
        if i == 0:
            word_tag_list[i].append('<START>')
        if i != len(word_tag_list)-1:
            if '.' in word_tag_list[i][1]:
                word_tag_list[i].append('<END>')
                if '.' not in word_tag_list[i+1][1]:
                    word_tag_list[i+1].append('<START>')
            else:
                if '<START>' not in word_tag_list[i] and '<END>' not in word_tag_list[i]:
                    word_tag_list[i].append('<MID>')
        if i == len(word_tag_list)-1:
            word_tag_list[i].append('<END>')
    return word_tag_list

In [273]:
tokened_word_tag = token(all_text2)

In [278]:
def replace_UNK(word_tag_list, threshold):
    dic = {}
    for i in word_tag_list:
        if i[0] in dic:
            dic[i[0]] = dic[i[0]] + 1
        else:
            dic[i[0]] = 1
    
    for j in word_tag_list:
        if dic[j[0]]<threshold:
            j[0] = 'UNK'
    return word_tag_list

In [326]:
pre_word_tag = split_word_tag(tokened_word_tag)
Replaced_word_tag = replace_UNK(pre_word_tag, 4) # Set threshold to 4 to replace words with too small frequencies
training_word_tag = add_START_END(Replaced_word_tag)

In [287]:
word_tag={}
tag_uni={}
tag_bi = {}
for i  in range(len(training_word_tag)):
    
    word_tag_ele = training_word_tag[i][0] + '|'+training_word_tag[i][1]
    if word_tag_ele in word_tag:
        word_tag[word_tag_ele] += 1
    if word_tag_ele not in word_tag:
        word_tag[word_tag_ele] = 1
        
    tag_uni_ele = training_word_tag[i][1]
    if tag_uni_ele in tag_uni:
        tag_uni[tag_uni_ele] +=1
    if tag_uni_ele not in tag_uni:
        tag_uni[tag_uni_ele] =1

    if i != 0:
        tag_bi_ele = training_word_tag[i][1] + '|' + training_word_tag[i-1][1] #ti, ti-1 form
        if tag_bi_ele in tag_bi:
            tag_bi[tag_bi_ele] +=1
        else:
            tag_bi[tag_bi_ele] =1

In [434]:
with open('Question_4_4.1_word-tag counts.txt', 'w') as f:
    for i, j in word_tag.items():
        f.writelines(i + ': ' + str(j) + '\n')
    f.close()

In [435]:
with open('Question_4_4.1_tag-unigram counts.txt', 'w') as f:
    for i, j in tag_uni.items():
        f.writelines(i + ': ' + str(j) + '\n')
    f.close()

In [437]:
with open('Question_4_4.1_tag-bigram counts.txt', 'w') as f:
    f.writelines('The tag-bigram has the form ti|ti-1\n')
    for i, j in tag_bi.items():
        f.writelines(i + ': ' + str(j) + '\n')
    f.close()

### 4.2

In [288]:
def transition_probability(tag_unigram,tag_bigram,tag_bigram_prob):
    for i in tag_bigram.keys():
        ti_1 = (i.split('|'))[1]
        tag_bigram_prob[i] = tag_bigram[i]/tag_uni[ti_1]
    return tag_bigram_prob

In [289]:
transition_probabilities = transition_probability(tag_uni, tag_bi, {})

In [439]:
with open('Question_4_4.2_transition probability.txt', 'w') as f:
    f.writelines('The tag-bigram has the form ti|ti-1, but it calculate P(ti−1, ti).\n')
    for i, j in transition_probabilities.items():
        f.writelines(i + ': ' + str(j) + '\n')
    f.close()

### 4.3

In [290]:
def emission_probability(tag_unigram,word_tag_dic,word_tag_prob):
    for i in word_tag_dic.keys():
        ti = (i.split('|'))[1]
        word_tag_prob[i] = word_tag_dic[i]/tag_uni[ti]
    return word_tag_prob

In [291]:
emission_probabilities = emission_probability(tag_uni, word_tag, {})

In [440]:
with open('Question_4_4.3_emission probability.txt', 'w') as f:

    for i, j in emission_probabilities.items():
        f.writelines(i + ': ' + str(j) + '\n')
    f.close()

### 4.4

In [292]:
Start_word = {}
Start_tag = {}
for i in range(len(training_word_tag)):
    if training_word_tag[i][-1]== '<START>':
        if training_word_tag[i][0]+'|'+training_word_tag[i][1] in Start_word:
            Start_word[training_word_tag[i][0]+'|'+training_word_tag[i][1]] += 1
        if training_word_tag[i][0]+'|'+training_word_tag[i][1] not in Start_word:
            Start_word[training_word_tag[i][0]+'|'+training_word_tag[i][1]] = 1
        
        if training_word_tag[i][1] in Start_tag:
            Start_tag[training_word_tag[i][1]] +=1
        if training_word_tag[i][1] not in Start_tag:
            Start_tag[training_word_tag[i][1]] =1

In [293]:
Start_tag_prob = {}
    
for n in Start_tag.keys():
    Start_tag_prob[n] = Start_tag[n]/sum(list(Start_tag.values()))

In [294]:
star_word_emission_prob = emission_probability(Start_tag, Start_word, {})

In [328]:
def mid_and_end_words(first_tag, transition_probability, emission_probability, max_iter, P_begin):
    previous_tag = first_tag
    WORD_output = []
    iter_ = 0
    while iter_ <= max_iter: #I don't want the sentence be to long who'll have a really small probability, therefore I
                             # set an max_iteration.
        candidate_tags = {}
        for i in transition_probabilities.keys():
            if i.split('|')[1] == previous_tag:
                candidate_tags[i.split('|')[0]] = transition_probabilities[i]
        tag_nomalizer = sum(list(candidate_tags.values()))
        TAG = np.random.choice(list(candidate_tags.keys()), 1, 
                               [k/tag_nomalizer for k in list(candidate_tags.values())])
        #print(TAG)
        candidate_word_tag = {}
        for j in emission_probability.keys():
            if j.split('|')[1] == TAG[0]:
                if j.split('|')[0] != '':
                    candidate_word_tag[j] = emission_probability[j]
        word_normalizer = sum(list(candidate_word_tag.values()))
        WORD = np.random.choice(list(candidate_word_tag.keys()), 1,
                                [l/word_normalizer for l in list(candidate_word_tag.values())])
        WORD_output.append(WORD[0])
        #print(TAG[0]+ '|') #+ previous_tag])
        P_begin += np.log(transition_probability[TAG[0] + '|'+ previous_tag]) + np.log(emission_probability[WORD[0]])
        
        previous_tag = TAG[0]
        if TAG == '.':
            break
        iter_ += 1
    return WORD_output, np.e ** P_begin

In [341]:
for i in range(5):
    p_begin = 0
    starter_tag = np.random.choice(list(Start_tag_prob.keys()), 1, list(Start_tag_prob.values()))
    candidate_starter_word = {}
    for j in star_word_emission_prob.keys():
        if j.split('|')[1] == starter_tag:
            #print(j)
            candidate_starter_word[j] = star_word_emission_prob[j]
    starter_normalizer = sum(list(candidate_starter_word.values()))
    starer_word = np.random.choice(list(candidate_starter_word.keys()), 1,
                                   [w/starter_normalizer for w in list(candidate_starter_word.values())])
    #print(starter_tag)
    p_begin = np.log(Start_tag_prob[starter_tag[0]]) + np.log(emission_probabilities[starer_word[0]])
    

    mid, p_begin = mid_and_end_words(starter_tag[0], transition_probabilities,
                                     emission_probabilities, 40, p_begin)
    sentence = [starer_word[0]] + mid

    print('The sentence is ' + str(sentence)+' with probability ' + str(p_begin) + '\n')

The sentence is ['he|pps', '29th|od', 'last|ap', 'whoever|wps', "she's|pps+bez", 'right|nn-tl', 'up|rp', ':|:', "he's|pps+bez", 'such|abl', 'kennings|nns', ')|)', "'|'", "don't|do*", ';|.'] with probability 1.4829323916154647e-58

The sentence is ['naturally|ql-hl', 'little|ap-hl', 'to|to-hl', 'be|be-hl', 'larger|jjr-hl', '.|.-hl', 'those|dts', 'whatever|wdt', 'oneself|ppl', '``|``', "where'd|wrb+md", 'ye|ppss', ':|.'] with probability 9.22193382424271e-43

The sentence is ['no|at', 'wednesday|nr', 'even|ql', 'apparently|rb', 'UNK|bed', 'mayor|nn-tl', 'gables|nps', 'any|dti', 'e|nn-tl', '400|cd', 'UNK|abn', 'ought|md', "let's|vb+ppo", 'adventure|vb', 'we|ppss-tl', 'may|md-tl', 'have|hv-tl', 'him|ppo-tl', 'commissions|nns-tl', 'beside|in', "who's|wps+bez-tl", 'what|wps-tl', 'writes|vbz-tl', ',|,', 'this|dt', 'actively|ql', 'UNK|wrb', 'am|bem', 'interferometer|nn', 'most|ql-tl', 'congolese|jj-tl', 'an|at', 'laughed|vbd', 'those|dts', 'were|bed', 'easier|jjr', 'subjectively|ql', 'upper|jj

### 4.4

In [423]:
tag_uniform_probability = {}
sum_tag = sum(list(tag_uni.values()))
for i in tag_uni:
     tag_uniform_probability[i] = tag_uni[i]/sum_tag
tag_uniform_probability = dict(sorted(tag_uniform_probability.items(), key=operator.itemgetter(1),reverse=True))


In [418]:
def viterbi(sentence, all_tag, start_prob, transition_prob, emission_prob):
    N = len(all_tag)
    T = len(sentence)
    path_probability_matrix = np.zeros((N,T))
    path = {}
    for s in range(N):
        # initialize path_probability_matrix
        if sentence[0]+'|' + all_tag[s] in emission_prob:
            path_probability_matrix[s][0] = start_prob[all_tag[s]] * emission_prob[sentence[0]+'|' + all_tag[s]]
        # initialize the path: each tag is possible to begin with
        path[all_tag[s]] = [all_tag[s]]
        
    for t in range(1,T):
        
        new_tag = {} #The new tag that will become the path
        for s in range(N):
            candicate_prob = [] # now try the previous tag
            for s1 in range(N):
                if sentence[t]+'|'+all_tag[s] in emission_prob and all_tag[s]+'|'+all_tag[s1] in transition_prob:
                    candicate_prob.append(path_probability_matrix[s1][t-1]
                                          *transition_prob[all_tag[s]+'|'+all_tag[s1]]
                                          *emission_prob[sentence[t]+'|'+all_tag[s]])
                else:
                    candicate_prob.append(0)
            path_probability_matrix[s][t] = max(candicate_prob) # return the maximum probability
            temp_tag = all_tag[np.argmax(candicate_prob)] # this previous tag gives the maximum probability
            new_tag[all_tag[s]] = path[temp_tag] + [all_tag[s]] # A new optimal previous tag has been selected
        path = new_tag  #ignore all other non-optimal previous tag, only update new one's 
    
    num = np.argmax(path_probability_matrix[:,T-1])
    best_tag_path = path[list(path.keys())[num]]
    return best_tag_path

In [397]:
f = open('Test_File.txt', mode = 'r')
test_txt = f.readlines()
f.close()

In [417]:
test_sentence = {}
output_sentence = {}
for i in test_txt:
    if 'sentence ID' in i:
        sentence_id = i[:-1]
        tokend_words = []
        output_words = []
    if '< EOS >' in i:
        test_sentence[sentence_id] = tokend_words
        output_sentence[sentence_id] = output_words
    else: 
        if 'sentence ID' not in i:
            x = i[:-1].lower()
            if x not in [k.split("|")[0] for k in list(emission_probabilities.keys())]:
                x = 'UNK'
            tokend_words.append(x)
            output_words.append(i[:-1])

In [427]:
with open('Question_4_4.5.txt', 'w') as f:
    for i in test_sentence.keys():
        f.writelines(i + '\n')
        tokened_sent = test_sentence[i]
        showed_sent = output_sentence[i]
        test_tags = viterbi(tokened_sent, list(tag_uniform_probability.keys()),tag_uniform_probability, 
                            transition_probabilities, emission_probabilities)
        for j in range(len(tokened_sent)):
            f.writelines(showed_sent[j] + '/'+test_tags[j]+'\n')
        f.writelines('< EOS >\n')